In [5]:
import sys
sys.path.append('./')
import torch
import numpy as np
from src.dmtet.dmtet import DMTetGeometry
from src.diffelastic.mesh import TetMesh
from src.diffelastic.deform import Deform
from src.utils.visualize import viewer

# res = 3
# num_iter = 1000
# DMTet = DMTetGeometry(res=res, iter=num_iter)
# verts, tets = DMTet.getMesh()
# tet_mesh = TetMesh(verts, tets) 
# viewer(verts.cpu().detach(), tets.cpu().detach(), draw_tet=True).show()

# 这里假设gt audio是全为0的声音
# gt_audio = torch.zeros((1, 1000)).cuda()
# loss_func = torch.nn.MSELoss()

# optimizer = torch.optim.Adam(DMTet.parameters(), lr=0.03)

# for i in range(100):
#     audio_loss, reg_loss = DMTet.tick(gt_audio, loss_func, i)
#     print("iter:{}, audio_loss: {}, reg_loss: {}".format(i, audio_loss, reg_loss))

#     total_loss = audio_loss + reg_loss
#     optimizer.zero_grad()
#     total_loss.backward()
#     optimizer.step()

# verts, tets = DMTet.getMesh()

verts, tets = torch.load("output/dmtet_0.pth")

viewer(verts.cpu().detach(), tets.cpu().detach(), draw_tet=True).show()


Box(children=(Box(children=(FigureWidget({
    'data': [{'i': array([  0,   0,   0, ..., 325, 349, 350]),
    …